In [39]:
pip install torch

Keyring is skipped due to an exception: 'keyring.backends'
  Using cached torch-1.13.1-cp37-cp37m-manylinux1_x86_64.whl (887.5 MB)
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl (557.1 MB)
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl (849 kB)
  Using cached nvidia_cublas_cu11-11.10.3.66-py3-none-manylinux1_x86_64.whl (317.1 MB)
  Using cached nvidia_cuda_nvrtc_cu11-11.7.99-2-py3-none-manylinux1_x86_64.whl (21.0 MB)
Note: you may need to restart the kernel to use updated packages.


In [31]:
pip install -U transformers evaluate datasets

Keyring is skipped due to an exception: 'keyring.backends'
Note: you may need to restart the kernel to use updated packages.


In [56]:
import pandas as pd
import numpy as np
import os
import json
import random
import evaluate
import torch
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer, pipeline
from sagemaker.huggingface.model import HuggingFaceModel
import transformers

### Preparing the CCase data

In [57]:
os.chdir('/root/nlp-coe/nltk_data/corpora/ccase/')
df_ccase=pd.read_csv('results.csv')
print(df_ccase.shape)
df_ccase.head()

(13, 3)


,agent_id,customerccase_id,ccases
0,XXXX0005,AAAA1114,"[{""M"":{""ccase_id"":{""S"":""00000048""},""filename"":..."
1,XXXX0005,AAAA1115,"[{""M"":{""ccase_id"":{""S"":""00000021""},""channel"":{..."
2,XXXX0005,AAAA1116,"[{""M"":{""ccase_id"":{""S"":""00000044""},""filename"":..."
3,XXXX0004,AAAA1116,"[{""M"":{""ccase_id"":{""S"":""00000045""},""filename"":..."
4,XXXX0004,AAAA1117,"[{""M"":{""ccase_id"":{""S"":""00000047""},""filename"":..."


In [58]:

json.loads(df_ccase['ccases'][0])[0]['M']['data']['M']['keyphrases']['L'][0]['L']

[{'S': 'Jessica speaks to Jessica about how she got her iPad back online'},
 {'S': 'Jessica says her iPad is connected automatically through WiFi Jessica: "I think Google stuff again'},
 {'S': "This is amazing I think Google things again That's great to hear"},
 {'S': 'Is there anything else I can help you with today? Nope'},
 {'S': 'Could I ask how satisfied you were with a resolution today? Very satisfied Very satisfied, very satisfied'}]

In [59]:
next_actions = [0, 1, 2]
# next_actions = ["DO NOTHING", "RECOMMEND PROMOTION", "PROVIDE REFUND"]

In [60]:
def prepare(ccase):
    d = json.loads(ccase)[0]
    output = '. '.join(a['S'] for a in d['M']['data']['M']['keyphrases']['L'][0]['L'])
    return output

In [61]:
df_ccase['summary'] = df_ccase['ccases'].apply(lambda x: prepare(x))
df_ccase['class_'] = random.choices(next_actions, k=len(df_ccase))

In [62]:
df_ccase

,agent_id,customerccase_id,ccases,summary,class_
0,XXXX0005,AAAA1114,"[{""M"":{""ccase_id"":{""S"":""00000048""},""filename"":...",Jessica speaks to Jessica about how she got he...,1
1,XXXX0005,AAAA1115,"[{""M"":{""ccase_id"":{""S"":""00000021""},""channel"":{...",Victoria Broadband has been struggling to find...,1
2,XXXX0005,AAAA1116,"[{""M"":{""ccase_id"":{""S"":""00000044""},""filename"":...",Jessica's iPad is connected automatically for ...,2
3,XXXX0004,AAAA1116,"[{""M"":{""ccase_id"":{""S"":""00000045""},""filename"":...",Jessicas We're calling you back to formula tha...,0
4,XXXX0004,AAAA1117,"[{""M"":{""ccase_id"":{""S"":""00000047""},""filename"":...",Jessica's iPad connected automatically for WiF...,1
5,XXXX0003,AAAA1114,"[{""M"":{""ccase_id"":{""S"":""00000010""},""channel"":{...",A large wave of orders came in last week So ou...,2
6,XXXX0002,AAAA1114,"[{""M"":{""ccase_id"":{""S"":""00000009""},""channel"":{...","The bar code is dru K R U K K , which means ""D...",0
7,XXXX0002,AAAA1115,"[{""M"":{""ccase_id"":{""S"":""00000011""},""channel"":{...",There is sometimes and they were saying no co...,2
8,XXXX0002,AAAA1117,"[{""M"":{""ccase_id"":{""S"":""00000017""},""channel"":{...",Telefonica's new broadband service has unlimit...,0
9,XXXX0001,AAAA1114,"[{""M"":{""ccase_id"":{""S"":""00000008""},""channel"":{...",V voucher for an Apples phone is a voucher for...,1


In [63]:
# def training(summary, class_):
#     return {'class': class_, 'summary': summary}

In [64]:
df_ccase.summary

0     Jessica speaks to Jessica about how she got he...
1     Victoria Broadband has been struggling to find...
2     Jessica's iPad is connected automatically for ...
3     Jessicas We're calling you back to formula tha...
4     Jessica's iPad connected automatically for WiF...
5     A large wave of orders came in last week So ou...
6     The bar code is dru K R U K K , which means "D...
7      There is sometimes and they were saying no co...
8     Telefonica's new broadband service has unlimit...
9     V voucher for an Apples phone is a voucher for...
10    Manager at victoria's broadband says he is hav...
11    Mrs [PII] was waiting for her call for some ti...
12    Telefonica com will pay a direct debit at the ...
Name: summary, dtype: object

In [65]:
# df_ccase["training"] = df_ccase.apply(lambda x: training(x.summary, x.class_), axis=1)

In [66]:
# df_ccase['training'][0]

### Converting to HuggingFace Dataset

In [67]:
from datasets import Dataset
import pandas as pd
df = df_ccase[['summary', 'class_']]
df.columns = ['text', "label"]

dataset = Dataset.from_pandas(df)

In [68]:
dataset[0]

{'text': 'Jessica speaks to Jessica about how she got her iPad back online. Jessica says her iPad is connected automatically through WiFi Jessica: "I think Google stuff again. This is amazing I think Google things again That\'s great to hear. Is there anything else I can help you with today? Nope. Could I ask how satisfied you were with a resolution today? Very satisfied Very satisfied, very satisfied',
 'label': 1}

### Loading Tokeniser

In [69]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", problem_type="multi_label_classification")

### Applying Tokeniser

In [70]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [71]:
df_ccase_tokenised = dataset.map(preprocess_function)

  0%|          | 0/13 [00:00<?, ?ex/s]

In [72]:
df_ccase_tokenised

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 13
})

### Loading Padder

In [73]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

### Loading Evaluater

In [74]:
import evaluate

accuracy = evaluate.load("accuracy")

### Computing Metrics

In [75]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

### Setting Labels

In [76]:
id2label = {0: "No further operation needed.", 1: "Offer a promotion to the customer.", 2: "Apply a discount to the price."}
label2id = {"No further operation needed.": 0, "Offer a promotion to the customer.": 1, "Apply a discount to the price.": 2}

### Loading Model

In [77]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=3, id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classi

### Training Model

In [86]:
training_args = TrainingArguments(
    output_dir="my_awesome_model_new",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=df_ccase_tokenised,
    eval_dataset=df_ccase_tokenised,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

## Note that the train/eval datasets are the same - this is a dummy to be refined

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 13
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch 

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.033400,0.615385


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 13
  Batch size = 1
Saving model checkpoint to my_awesome_model_new/checkpoint-13
Configuration saved in my_awesome_model_new/checkpoint-13/config.json
Model weights saved in my_awesome_model_new/checkpoint-13/pytorch_model.bin
tokenizer config file saved in my_awesome_model_new/checkpoint-13/tokenizer_config.json
Special tokens file saved in my_awesome_model_new/checkpoint-13/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from my_awesome_model_new/checkpoint-13 (score: 1.0333997011184692).


TrainOutput(global_step=13, training_loss=1.0456160765427809, metrics={'train_runtime': 9.8473, 'train_samples_per_second': 1.32, 'train_steps_per_second': 1.32, 'total_flos': 282274432182.0, 'train_loss': 1.0456160765427809, 'epoch': 1.0})

### Testing Model

In [79]:
### Make this a dictionary

text = "This was a masterpiece. Not completely faithful to the books, but enthralling from beginning to end. Might be my favorite of the three."

In [80]:
input_text = [{'S': 'Jessica speaks to Jessica about how she got her iPad back online'},
 {'S': 'Jessica says her iPad is connected automatically through WiFi Jessica: "I think Google stuff again'},
 {'S': "This is amazing I think Google things again That's great to hear"},
 {'S': 'Is there anything else I can help you with today? Nope'},
 {'S': 'Could I ask how satisfied you were with a resolution today? Very satisfied Very satisfied, very satisfied'}]

In [81]:
processed_text = '. '.join(a["S"] for a in input_text)

In [82]:
processed_text

'Jessica speaks to Jessica about how she got her iPad back online. Jessica says her iPad is connected automatically through WiFi Jessica: "I think Google stuff again. This is amazing I think Google things again That\'s great to hear. Is there anything else I can help you with today? Nope. Could I ask how satisfied you were with a resolution today? Very satisfied Very satisfied, very satisfied'

In [83]:
os.chdir('/root/nlp-coe/nlp_models/models/Text Classification/')

In [84]:
output = '. '.join(a['S'] for a in d['M']['data']['M']['keyphrases']['L'][0]['L'])

NameError: name 'd' is not defined

In [87]:
classifier = pipeline("text-classification", model="my_awesome_model_new/checkpoint-13")
classifier(processed_text)

loading configuration file my_awesome_model_new/checkpoint-13/config.json
Model config DistilBertConfig {
  "_name_or_path": "my_awesome_model_new/checkpoint-13",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "No further operation needed.",
    "1": "Offer a promotion to the customer.",
    "2": "Apply a discount to the price."
  },
  "initializer_range": 0.02,
  "label2id": {
    "Apply a discount to the price.": 2,
    "No further operation needed.": 0,
    "Offer a promotion to the customer.": 1
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.26.1"

[{'label': 'Apply a discount to the price.', 'score': 0.3546299338340759}]

### Exporting Model

In [35]:
# !cd "my_awesome_model"

In [93]:
os.chdir('/root/nlp-coe/nlp_models/models/Text Classification/my_awesome_model_new/checkpoint-13/')

In [94]:
!ls

config.json	   scheduler.pt		    trainer_state.json
optimizer.pt	   special_tokens_map.json  training_args.bin
pytorch_model.bin  tokenizer.json	    vocab.txt
rng_state.pth	   tokenizer_config.json


In [95]:
!tar zcvf model.tar.gz *

config.json
optimizer.pt
pytorch_model.bin
rng_state.pth
scheduler.pt
special_tokens_map.json
tokenizer.json
tokenizer_config.json
trainer_state.json
training_args.bin
vocab.txt


In [97]:
!aws s3 cp model.tar.gz s3://aws-nlp-coe/models/ccase_models/

upload: ./model.tar.gz to s3://aws-nlp-coe/models/ccase_models/model.tar.gz


In [98]:
huggingface_model = HuggingFaceModel(
   model_data="s3://aws-nlp-coe/models/ccase_models/model.tar.gz",  # path to your trained SageMaker model
   role=role,                                            # IAM role with permissions to create an endpoint
   transformers_version="4.17.0",                           # Transformers version used
   pytorch_version="1.10.2",                                # PyTorch version used
   py_version='py38',                                    # Python version used
)

In [100]:
huggingface_model

In [101]:
huggingface_model.create(instance_type='ml.g4dn.xlarge')

In [102]:
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.g4dn.xlarge"
)




-----!

In [103]:
processed_text

'Jessica speaks to Jessica about how she got her iPad back online. Jessica says her iPad is connected automatically through WiFi Jessica: "I think Google stuff again. This is amazing I think Google things again That\'s great to hear. Is there anything else I can help you with today? Nope. Could I ask how satisfied you were with a resolution today? Very satisfied Very satisfied, very satisfied'

In [105]:
output = {"inputs": processed_text}
output

{'inputs': 'Jessica speaks to Jessica about how she got her iPad back online. Jessica says her iPad is connected automatically through WiFi Jessica: "I think Google stuff again. This is amazing I think Google things again That\'s great to hear. Is there anything else I can help you with today? Nope. Could I ask how satisfied you were with a resolution today? Very satisfied Very satisfied, very satisfied'}

In [ ]:
s3://aws-nlp-coe/models/ccase_models/

In [113]:
huggingface_model

In [122]:
batch_job = huggingface_model.transformer(instance_count = 1, instance_type = 'ml.g4dn.xlarge', strategy='SingleRecord'
)

In [131]:
bucketsess = sagemaker.Session()
role=sagemaker.get_execution_role()
sagemaker_session_bucket = sess.default_bucket()

In [132]:
sagemaker_session_bucket

'sagemaker-eu-west-1-866647877267'

In [125]:
batch_job.transform(data='s3://aws-nlp-coe/models/ccase_models/input.jsonl', content_type='application/json', split_type='Line'
)

2023-02-13T17:12:42,291 [INFO ] main com.amazonaws.ml.mms.ModelServer - 
MMS Home: /opt/conda/lib/python3.8/site-packages
Current directory: /
Temp directory: /home/model-server/tmp
Number of GPUs: 1
Number of CPUs: 4
Max heap size: 3499 M
Python executable: /opt/conda/bin/python3.8
Config file: /etc/sagemaker-mms.properties
Inference address: http://0.0.0.0:8080
Management address: http://0.0.0.0:8080
Model Store: /.sagemaker/mms/models
Initial Models: ALL
Log dir: null
Metrics dir: null
Netty threads: 0
Netty client threads: 0
Default workers per model: 1
Blacklist Regex: N/A
Maximum Response Size: 6553500
Maximum Request Size: 6553500
Preload model: false
Prefer direct buffer: false
2023-02-13T17:12:42,366 [WARN ] W-9000-model com.amazonaws.ml.mms.wlm.WorkerLifeCycle - attachIOStreams() threadName=W-9000-model
2023-02-13T17:12:42,451 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - model_service_worker started with args: --sock-type unix --sock-name /home/model

In [107]:
# create transformer to run a batch job
batch_job = huggingface_model.transformer(
    instance_count=1,
    instance_type='ml.g4dn.xlarge',
    strategy='SingleRecord'
)

# starts batch transform job and uses S3 data as input
batch_job.transform(
    data='s3://aws-nlp-coe/models/ccase_models/input.jsonl',
    content_type='application/json',    
    split_type='Line'
)

SyntaxError: invalid character in identifier (<ipython-input-107-8699d61fefa9>, line 3)

In [ ]:
data = {
"inputs": {
	"question": "What is used for inference?",
	"context": "My Name is Philipp and I live in Nuremberg. This model is used with sagemaker for inference."
	}
}

In [41]:
import sagemaker
sess = sagemaker.Session()
role = sagemaker.get_execution_role()

In [45]:
!python --version

Python 3.7.10


In [49]:
torch.__version__

'1.13.1+cu117'

In [52]:
transformers.__version__

'4.26.1'

In [ ]:
huggingface_model = HuggingFaceModel(
   model_data="s3://models/my-bert-model/model.tar.gz",  # path to your trained SageMaker model
   role=role,                                            # IAM role with permissions to create an endpoint
   transformers_version="4.6",                           # Transformers version used
   pytorch_version="1.7",                                # PyTorch version used
   py_version='py36',                                    # Python version used
)